In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv('./Datasets/insurance_data.csv')
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [5]:
X = df[['age', 'affordibility']]
y = df['bought_insurance']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)
print(len(X_test))

6


In [6]:
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age']/100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age']/100

In [7]:
X_train_scaled

,age,affordibility
8,0.62,1
24,0.50,1
19,0.18,1
0,0.22,1
27,0.46,1
4,0.46,1
11,0.28,1
9,0.61,1
25,0.54,1
13,0.29,0


In [8]:
def log_loss(y_true, y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i,epsilon) for i in y_predicted]
    y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

In [9]:
def sigmoid_numpy(X):
    return 1/(1 + np.exp(-X))

sigmoid_numpy(np.array([12, 0, 1]))

array([0.99999386, 0.5       , 0.73105858])

In [16]:
class neural_network:
    def __init__(self):
        self.w1 = 1
        self.w2 = 1
        self.bias = 0

    # gradient descent method
    def gradient_descent(self, age, affordibility, y_true, epochs, loss_threshold):
    # we try to come up with best values for w1, w2, bias
        w1 = w2 = 1
        bias = 0
        learning_rate = 0.5

        n = len(age)
        for i in range(epochs):
            weighted_sum = w1*age + w2*affordibility + bias
            y_predicted = sigmoid_numpy(weighted_sum)

            loss = log_loss(y_true, y_predicted)

            w1d = (1/n)*np.dot(np.transpose(age),(y_predicted - y_true)) 
            w2d = (1/n)*np.dot(np.transpose(affordibility),(y_predicted - y_true)) 
            bias_d = np.mean(y_predicted - y_true)

            w1 = w1 - learning_rate*w1d
            w2 = w2 - learning_rate*w2d
            bias = bias - learning_rate*bias_d

            if i%50 == 0:
                print(f"Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}")

            if loss<=loss_threshold:
                print(f"Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}")
                break

        return w1, w2, bias

    # fit method
    def fit(self, X, y, epochs, loss_threshold):
        self.w1, self.w2, self.bias = self.gradient_descent(X['age'], X['affordibility'], y, epochs, loss_threshold)

    def predict(self, X_test):
        weighted_sum = X_test['age']*self.w1 + X_test['affordibility']*self.w2 + self.bias
        return sigmoid_numpy(weighted_sum)

In [17]:
model = neural_network()
model.fit(X_train_scaled, y_train, epochs=500, loss_threshold=0.4631)

Epoch:0, w1:0.9651327599336419, w2:0.9132156721803109, bias:-0.14382765768900405, loss:0.7891487079051143
Epoch:50, w1:1.47087282368346, w2:0.9668858470813096, bias:-1.3762453752550694, loss:0.5769284463195493
Epoch:100, w1:2.159902298320211, w2:1.2083273283369045, bias:-1.855164884834696, loss:0.5460946210611893
Epoch:150, w1:2.801143422604105, w2:1.3381805431216727, bias:-2.221410133020174, loss:0.5234671972165441
Epoch:200, w1:3.3899097838026924, w2:1.4147453459757593, bias:-2.521317389764026, loss:0.5056867627954232
Epoch:250, w1:3.9275040482519703, w2:1.464671227127734, bias:-2.7779770795357766, loss:0.49133388311503035
Epoch:300, w1:4.417546684162967, w2:1.5009244863121816, bias:-3.003979249687541, loss:0.4795886770199994
Epoch:350, w1:4.864402872277743, w2:1.530029445571582, bias:-3.2067167593587964, loss:0.4698891497619659
Epoch:392, w1:5.209615079180804, w2:1.5514415804090527, bias:-3.362484584079228, loss:0.4630151380127964


In [18]:
model.predict(X_test_scaled)

21    0.118367
15    0.741589
17    0.770393
1     0.113038
7     0.441096
23    0.630249
dtype: float64